In [1]:
from fpl_helpers import *
import pandas as pd

In [2]:
elements_df, teams_df = get_tables()
global PLAYER_IDS
PLAYER_IDS = elements_df[['full_name', 'id']].set_index('full_name').to_dict()['id']
PLAYER_NAMES = {PLAYER_IDS[i]:i for i in list(PLAYER_IDS.keys()) }


In [3]:
def get_team(team_id, gw):
  url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/"
  r = requests.get(url)
  json = r.json()
  return json

team_ids = [i['element'] for i in get_team(715692, 25)['picks']]
team_names = [PLAYER_NAMES[i] for i in team_ids]

In [4]:
team_names

['Jordan Pickford',
 'Daniel Muñoz',
 'Gabriel dos Santos Magalhães',
 'Andrew Robertson',
 'Morgan Rogers',
 'Mohamed Salah',
 'Anthony Gordon',
 'Cole Palmer',
 'Enzo Fernández',
 'Chris Wood',
 'Ollie Watkins',
 'Hákon Valdimarsson',
 'Cody Gakpo',
 'Lewis Hall',
 'Taylor Harwood-Bellis']

In [5]:
compare_players(team_names[5:11], metrics=ATTACKER_METRICS, agg='mean')

,Difficulty,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected
Mohamed Salah,"4.0, 3.0, 2.0, 2.0, 3.0, 2.0",11.00,1.00,0.52,0.67,0.08,0.60,2.00,37.33,50.13,22.20,35.33,10.77,1.00,8092986.33
Anthony Gordon,"3.0, 5.0, 2.0, 2.0, 2.0, 3.0",2.67,0.00,0.05,0.33,0.12,0.18,0.00,9.33,9.87,23.17,11.00,4.40,1.00,2478450.33
Cole Palmer,"3.0, 2.0, 2.0, 4.0, 3.0, 3.0",3.33,0.00,0.17,0.33,0.25,0.42,0.67,18.67,8.00,20.83,18.00,4.70,1.00,7271425.67
Enzo Fernández,"3.0, 2.0, 2.0, 4.0, 3.0, 3.0",2.33,0.00,0.21,0.33,0.04,0.25,0.00,12.33,8.53,12.53,7.00,2.83,1.00,457696.00
Chris Wood,"4.0, 4.0, 4.0, 2.0, 3.0, 3.0",9.33,1.33,0.77,0.00,0.01,0.78,2.00,41.00,48.20,12.60,39.67,10.07,1.00,4824478.33
Ollie Watkins,"4.0, 3.0, 3.0, 3.0, 3.0, 2.0",5.33,0.67,0.33,0.00,0.04,0.38,1.00,18.67,22.80,4.00,20.00,4.67,1.00,2189490.33


In [6]:
forwards = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==4)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
forwards['difficulty'] = forwards['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
forwards['avg_difficulty'] = forwards['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
forwards['cost_per_point'] = forwards['now_cost']/forwards['total_points']
#forwards[:20].sort_values(by='difficulty')
forwards

,full_name,team,selected_by_percent,now_cost,value_season,total_points,expected_goals,goals_scored,expected_assists,assists,expected_goals_conceded,goals_conceded,difficulty,avg_difficulty,cost_per_point
602,Chris Wood,Nott'm Forest,44.8,71,23.0,163,10.31,18,0.38,2,24.93,25,"[4.0, 4.0, 4.0, 2.0, 3.0, 3.0]",3.333333,0.435583
475,Erling Haaland,Man City,27.4,147,10.5,154,18.30,19,1.66,3,35.00,35,"[5.0, 3.0, 4.0, 3.0, 2.0, 3.0]",3.333333,0.954545
553,Alexander Isak,Newcastle,57.9,95,16.2,154,14.12,17,3.11,5,26.77,31,"[3.0, 5.0, 2.0, 2.0, 2.0, 3.0]",2.833333,0.616883
743,Matheus Santos Carneiro Da Cunha,Wolves,9.3,68,19.4,132,5.76,12,4.12,4,35.31,47,"[3.0, 3.0, 2.0, 2.0, 2.0, 2.0]",2.333333,0.515152
64,Ollie Watkins,Aston Villa,20.4,89,14.7,131,12.24,12,1.52,5,25.10,32,"[4.0, 3.0, 3.0, 3.0, 3.0, 2.0]",3.000000,0.679389
266,Jean-Philippe Mateta,Crystal Palace,8.6,73,15.1,110,10.64,11,2.06,1,29.77,27,"[3.0, 3.0, 2.0, 2.0, 3.0, 4.0]",2.833333,0.663636
232,Nicolas Jackson,Chelsea,7.7,77,14.0,108,10.71,9,1.65,6,23.95,19,"[3.0, 2.0, 2.0, 4.0, 3.0, 3.0]",2.833333,0.712963
338,Raúl Jiménez,Fulham,8.5,56,18.8,105,9.14,9,1.65,3,19.62,22,"[3.0, 2.0, 3.0, 3.0, 4.0, 5.0]",3.333333,0.533333
143,Yoane Wissa,Brentford,12.3,63,16.7,105,11.46,10,1.25,3,29.93,30,"[2.0, 2.0, 3.0, 3.0, 4.0, 4.0]",3.000000,0.600000
363,Liam Delap,Ipswich,3.6,56,18.0,101,7.58,10,1.26,2,38.48,38,"[3.0, 3.0, 3.0, 3.0, 3.0, 2.0]",2.833333,0.554455


In [7]:
forward_scores = compare_players(forwards['full_name'], metrics=ATTACKER_METRICS, agg='mean')
forward_scores.data['Average Difficulty'] = forward_scores.data['Difficulty'].apply(lambda x: np.mean(x))
#forward_scores

In [8]:
preferred_low = ["Average Difficulty"]

In [9]:
weights = {
    'Average Difficulty':-2,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':3,
    'Assists':0,
    'Expected Assists':0, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':10
}
df = calculate_weighted_score(forward_scores.data.drop('Difficulty', axis=1), weights, preferred_low)
df

,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Jean-Philippe Mateta,7.33,1.00,0.89,0.00,0.28,1.17,1.33,31.33,42.20,16.50,35.67,9.43,1.00,729543.33,2.83,17.65
Chris Wood,9.33,1.33,0.77,0.00,0.01,0.78,2.00,41.00,48.20,12.60,39.67,10.07,1.00,4824478.33,3.33,17.47
Norberto Bercique Gomes Betuncal,9.67,1.33,0.60,0.00,0.02,0.62,2.33,40.33,53.27,18.77,44.67,11.70,1.00,119024.67,3.17,17.36
Matheus Santos Carneiro Da Cunha,6.00,0.67,0.35,0.00,0.15,0.50,1.67,26.67,32.40,19.00,35.33,8.67,1.00,891656.67,2.33,15.59
Alexander Isak,5.67,0.67,0.53,0.00,0.15,0.68,1.00,25.67,28.73,13.47,32.33,7.47,1.00,6715710.00,2.83,14.82
Liam Delap,6.00,0.67,0.37,0.00,0.09,0.46,1.67,20.67,24.13,10.57,26.33,6.03,1.00,380673.67,2.83,14.42
Erling Haaland,7.67,0.67,0.29,0.67,0.14,0.43,1.00,33.33,36.20,18.43,34.67,8.93,1.00,3168319.67,3.33,13.39
Ollie Watkins,5.33,0.67,0.33,0.00,0.04,0.38,1.00,18.67,22.80,4.00,20.00,4.67,1.00,2189490.33,3.00,13.28
Jamie Vardy,4.00,0.33,0.38,0.00,0.02,0.40,1.00,17.67,17.27,5.07,17.00,3.93,1.00,628162.67,3.00,12.70
Raúl Jiménez,5.00,0.33,0.35,0.33,0.04,0.39,0.67,18.33,20.33,20.03,43.67,8.33,1.00,1048572.00,3.33,11.92


In [10]:
midfielders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==3)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
midfielders['difficulty'] = midfielders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
midfielders['avg_difficulty'] = midfielders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
midfielders['cost_per_point'] = midfielders['now_cost']/midfielders['total_points']
#forwards[:20].sort_values(by='difficulty')
#midfielders

In [14]:
mid_weights = {
    'Average Difficulty': -3,
    'Total Points':1.5, 
    'Goals Scored': 2,
    'Expected Goals':2,
    'Assists':1,
    'Expected Assists':1, 
    'Expected Goal Involvements':1, 
    'Bonus':1.5,
    'Bps':1.5,
    'Influence':0, 
    'Creativity':0, 
    'Threat': 0,
    'Ict Index':0, 
    'Starts':1
}

midfielder_scores = compare_players(midfielders['full_name'], metrics=ATTACKER_METRICS, agg='mean')
midfielder_scores.data['Average Difficulty'] = midfielder_scores.data['Difficulty'].apply(lambda x: np.mean(x))
df = calculate_weighted_score(midfielder_scores.data.drop('Difficulty', axis=1), mid_weights, preferred_low)
df


,Total Points,Goals Scored,Expected Goals,Assists,Expected Assists,Expected Goal Involvements,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Selected,Average Difficulty,total_score
Mohamed Salah,11.00,1.00,0.52,0.67,0.08,0.60,2.00,37.33,50.13,22.20,35.33,10.77,1.00,8092986.33,2.67,9.38
Dango Ouattara,10.00,1.33,0.57,0.00,0.09,0.66,1.00,31.67,53.33,14.57,50.00,11.80,1.00,154676.33,2.33,9.35
Justin Kluivert,6.67,0.33,0.29,0.67,0.18,0.47,0.67,24.67,33.07,31.27,27.33,9.17,1.00,1865632.00,2.33,6.59
Antoine Semenyo,5.00,0.33,0.38,0.33,0.11,0.49,0.33,18.67,35.13,23.00,53.33,11.13,1.00,803473.33,2.33,5.68
Kevin Schade,7.33,0.67,0.36,0.00,0.07,0.43,1.33,22.00,28.40,13.60,47.67,8.97,1.00,70160.67,3.00,4.79
Bryan Mbeumo,4.00,0.33,0.59,0.00,0.22,0.81,0.00,18.33,21.53,25.93,32.00,7.93,1.00,3855887.00,3.00,4.31
Noni Madueke,4.00,0.33,0.35,0.00,0.08,0.43,0.67,14.00,17.27,20.40,15.33,5.30,1.00,492087.33,2.83,3.53
Morgan Gibbs-White,7.67,0.33,0.16,1.00,0.08,0.25,0.67,30.67,25.53,25.93,21.33,7.30,1.00,516607.33,3.33,3.34
Cole Palmer,3.33,0.00,0.17,0.33,0.25,0.42,0.67,18.67,8.00,20.83,18.00,4.70,1.00,7271425.67,2.83,3.25
Jacob Murphy,5.33,0.33,0.07,0.33,0.12,0.19,0.67,20.67,24.40,20.17,12.00,5.63,1.00,528826.00,2.83,3.17


In [12]:
defenders = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==2)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(30)
defenders['difficulty'] = defenders['full_name'].apply(lambda x: difficulty_range(PLAYER_IDS[x], gwrange=1))
defenders['avg_difficulty'] = defenders['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
defenders['cost_per_point'] = defenders['now_cost']/defenders['total_points']
defender_scores = compare_players(defenders['full_name'], metrics=DEFENDER_METRICS, agg='mean')
defender_scores.data['Average Difficulty'] = defender_scores.data['Difficulty'].apply(lambda x: np.mean(x))


In [13]:
def_weights ={
 'Average Difficulty': -2,
 'Total Points': 3,
 'Clean Sheets': 1,
 'Goals Conceded': -1,
 'Own Goals': -1,
 'Yellow Cards': -1,
 'Bonus': 1.5,
 'Bps': 1,
 'Influence': 1,
 'Creativity': 1,
 'Threat': 1,
 'Ict Index': 1,
 'Starts': 1,
 'Expected Goal Involvements': 1.5,
 'Expected Goals Conceded': -1.5
 }

df = calculate_weighted_score(defender_scores.data.drop('Difficulty', axis=1), def_weights)
df

,Total Points,Minutes,Clean Sheets,Goals Conceded,Own Goals,Yellow Cards,Red Cards,Bonus,Bps,Influence,Creativity,Threat,Ict Index,Starts,Expected Goal Involvements,Expected Goals Conceded,Average Difficulty,total_score
Pedro Porro,6.00,90.00,0.67,0.67,0.00,0.00,0.00,0.67,26.00,34.73,31.60,10.33,7.67,1.00,0.29,1.71,3.00,7.54
Daniel Muñoz,5.33,90.00,0.33,1.33,0.00,0.00,0.00,0.67,22.67,27.13,16.00,11.00,5.40,1.00,0.36,1.20,2.83,6.40
James Tarkowski,6.33,90.00,0.33,1.00,0.00,0.00,0.00,0.33,24.67,44.87,7.83,8.33,6.13,1.00,0.13,0.90,3.17,5.77
Nathan Collins,5.33,90.00,0.33,1.00,0.00,0.00,0.00,1.33,18.33,22.47,9.77,5.33,3.77,1.00,0.03,0.92,3.00,5.22
William Saliba,5.67,90.00,0.67,0.33,0.00,0.00,0.00,1.00,22.00,16.80,0.80,0.00,1.77,1.00,0.00,0.59,3.17,4.98
Gabriel dos Santos Magalhães,5.00,90.00,0.67,0.33,0.00,0.00,0.00,0.33,22.33,17.47,9.13,4.67,3.13,1.00,0.06,0.59,3.17,4.71
Joško Gvardiol,4.67,88.67,0.33,2.00,0.00,0.00,0.00,0.00,14.33,26.33,7.87,30.00,6.47,1.00,0.45,1.09,3.33,4.58
Trent Alexander-Arnold,4.67,52.33,0.33,1.00,0.00,0.00,0.00,0.00,13.33,23.40,17.80,7.33,4.83,0.67,0.13,0.68,2.67,4.56
Rayan Aït-Nouri,3.33,82.67,0.33,1.00,0.00,0.00,0.00,0.33,15.00,15.80,11.43,8.33,3.53,1.00,0.06,0.95,2.33,4.30
Lisandro Martínez,5.67,57.00,0.33,0.33,0.00,0.00,0.00,1.00,18.00,19.93,1.90,5.33,2.70,0.67,0.06,0.90,3.17,4.30
